In [3]:
pip install kagglehub

Note: you may need to restart the kernel to use updated packages.


In [7]:
pip install os

Note: you may need to restart the kernel to use updated packages.


ERROR: Could not find a version that satisfies the requirement os (from versions: none)
ERROR: No matching distribution found for os


In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.cluster import KMeans

In [5]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("vjchoudhary7/customer-segmentation-tutorial-in-python")

print("Path to dataset files:", path)

100%|██████████████████████████████████████████████████████████████████████████████| 1.55k/1.55k [00:00<00:00, 793kB/s]

Extracting files...
Path to dataset files: C:\Users\Dell\.cache\kagglehub\datasets\vjchoudhary7\customer-segmentation-tutorial-in-python\versions\1


In [9]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("vjchoudhary7/customer-segmentation-tutorial-in-python")

print("Path to dataset files:", path)

Path to dataset files: C:\Users\Dell\.cache\kagglehub\datasets\vjchoudhary7\customer-segmentation-tutorial-in-python\versions\1


In [11]:
df = pd.read_csv(path)


FileNotFoundError: [Errno 2] No such file or directory: '/root/.cache/kagglehub/datasets/vjchoudhary7/customer-segmentation-tutorial-in-python/versions/1/Mall_Customers.csv'

In [ ]:
df

In [ ]:
df.shape

In [ ]:
print('\nData types of the columns:')
print(df.dtypes)

In [ ]:
print(df.isnull().sum())


In [ ]:
df.describe().astype(int)


In [ ]:
sns.set(style='whitegrid')

fig, axs = plt.subplots(1, 3, figsize=(20, 5))

sns.histplot(data=df, x='Age', kde=True, color='blue', ax=axs[0])
sns.histplot(data=df, x='Annual Income (k$)', kde=True, color='green', ax=axs[1])
sns.histplot(data=df, x='Spending Score (1-100)', kde=True, color='red', ax=axs[2])

axs[0].set_title('Age Distribution')
axs[1].set_title('Annual Income Distribution')
axs[2].set_title('Spending Score Distribution')

fig.suptitle('Distribution Analysis of Age, Annual Income, and Spending Score')

plt.show()

In [ ]:
features = df[['Age', 'Annual Income (k$)', 'Spending Score (1-100)']]

# Determine the optimal number of clusters using the elbow method
wcss = []
for i in range(1, 11):
    kmeans = KMeans(n_clusters=i, init='k-means++', max_iter=300, n_init=10, random_state=0)
    kmeans.fit(features)
    wcss.append(kmeans.inertia_)

In [ ]:
plt.plot(range(1, 11), wcss)
plt.title('Elbow Method')
plt.xlabel('Number of clusters')
plt.ylabel('WCSS')
plt.show()

In [ ]:
kmeans = KMeans(n_clusters=5, init='k-means++', max_iter=300, n_init=10, random_state=0)

labels = kmeans.fit_predict(features)

df['Cluster'] = labels

df.head(100)



In [ ]:
cluster_means = df.groupby('Cluster')[['Age', 'Annual Income (k$)', 'Spending Score (1-100)']].mean().astype(int)

cluster_means

In [ ]:
plt.figure(figsize=(10, 8))
sns.scatterplot(data=df, x='Annual Income (k$)', y='Spending Score (1-100)', hue='Cluster', palette='viridis', s=100)

plt.title('Clusters of customers')

plt.show()

In [ ]:
plt.figure(figsize=(10, 8))
sns.scatterplot(data=df, x='Age', y='Spending Score (1-100)', hue='Cluster',
palette='viridis', s=100)

plt.title('Clusters of customers based on Age and Spending Score')

plt.show()

In [ ]:
plt.figure(figsize=(10, 6))
sns.countplot(data=df, x='Cluster', hue='Gender')

plt.title('Distribution of Gender within each Cluster')

plt.show()


In [ ]:
sns.pairplot(df, hue='Cluster', palette='viridis')
plt.show()


In [ ]:
fig, axs = plt.subplots(1, 3, figsize=(20, 5))

sns.boxplot(data=df, x='Cluster', y='Age', ax=axs[0])
axs[0].set_title('Age')

sns.boxplot(data=df, x='Cluster', y='Annual Income (k$)', ax=axs[1])
axs[1].set_title('Annual Income (k$)')

sns.boxplot(data=df, x='Cluster', y='Spending Score (1-100)', ax=axs[2])
axs[2].set_title('Spending Score (1-100)')

plt.show()

In [ ]:
from sklearn.metrics import silhouette_score

#The silhouette score is a measure of how well each datapoint lies within its cluster. It’s a way to quantify the quality of the clustering. The score ranges from -1 to 1.

silhouette_score = silhouette_score(df[['Annual Income (k$)', 'Spending Score (1-100)', 'Age']], df['Cluster'])
print('Silhouette Score: ', silhouette_score)


In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import silhouette_score
from sklearn.cluster import KMeans

scaler = StandardScaler()
scaled_features = scaler.fit_transform(df[['Annual Income (k$)', 'Spending Score (1-100)', 'Age']])



In [ ]:
kmeans = KMeans(n_clusters=5, random_state=42)
kmeans.fit(scaled_features)
df['Scaled_Cluster'] = kmeans.labels_

scaled_silhouette_score = silhouette_score(scaled_features, df['Scaled_Cluster'])
print('Scaled Silhouette Score:', scaled_silhouette_score)

In [ ]:
from sklearn.cluster import AgglomerativeClustering

agg_clustering = AgglomerativeClustering(n_clusters=5)
agg_clustering.fit(scaled_features)
df['Agg_Cluster'] = agg_clustering.labels_

agg_silhouette_score = silhouette_score(scaled_features, df['Agg_Cluster'])
print('Agglomerative Clustering Silhouette Score: ', agg_silhouette_score)

In [ ]:
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

pca = PCA(n_components=2)
pca_features = pca.fit_transform(scaled_features)

kmeans = KMeans(n_clusters=5, random_state=42)
kmeans.fit(pca_features)
df['PCA_Cluster'] = kmeans.labels_

pca_silhouette_score = silhouette_score(pca_features, df['PCA_Cluster'])
print('PCA Silhouette Score: ', pca_silhouette_score)

In [ ]:
from sklearn.cluster import DBSCAN

dbscan = DBSCAN(eps=0.5, min_samples=5)
dbscan.fit(pca_features)

df['DBSCAN_Cluster'] = dbscan.labels_

dbscan_silhouette_score = silhouette_score(pca_features, df['DBSCAN_Cluster'])
print('DBSCAN Silhouette Score: ', dbscan_silhouette_score)

In [ ]:
cluster_characteristics = df.groupby('Cluster')[['Age', 'Annual Income (k$)', 'Spending Score (1-100)']].mean().astype(int)

cluster_characteristics


In [ ]:
df

In [ ]:
'''
Cluster 0: “Conservative Middle-Aged”
Cluster 1: “Young High Earners”
Cluster 2: “Balanced Middle-Aged”
Cluster 3: “Wealthy Savers”
Cluster 4: “Young Spenders”
'''